In [1]:
import numpy as np
import pandas as pd

In [35]:
df = pd.read_csv('covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [37]:
from sklearn.model_selection import train_test_split

In [39]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['has_covid']),
                                                   df['has_covid'],test_size=0.2)

# 1. Aam Zindagi

In [42]:
# Adding simple imputer to fever column
from sklearn.impute import SimpleImputer
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])

# also the test data
X_test_fever = si.fit_transform(X_test[['fever']])


In [44]:
# ordinalencoding -- cough
from sklearn.preprocessing import OrdinalEncoder
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit_transform(X_train[['cough']])

# also the test data
X_test_cough = oe.fit_transform(X_test[['cough']])

In [54]:
# OHE -- gender, city
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(drop='first')
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])
X_train_gender_city.shape

(80, 4)

In [48]:
# Extracting Age
X_train_age = X_train.drop(columns=['gender','fever','cough','city']).values
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values
X_train_age.shape

(80, 1)

In [62]:
X_train_transformed = np.concatenate((X_train_age,X_train_fever,X_train_gender_city,X_train_cough),axis=1)

X_train_transformed = np.concatenate((X_test_age,X_test_fever,X_test_gender_city,X_test_cough),axis=1)



ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 2 has 0 dimension(s)

# Mentos Zindagi

In [65]:
from sklearn.compose import ColumnTransformer

In [71]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(drop='first'),['gender','city'])
],remainder='passthrough')

In [75]:
transformer.fit_transform(X_train).shape

(80, 7)

In [77]:
transformer.transform(X_test).shape

(20, 7)

In [81]:
# Example

import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

# Step 1: Create DataFrame
df = pd.DataFrame({
    'Name': ['Amit', 'Priya', 'Raj', 'Sneha'],
    'Department': ['HR', 'IT', 'Finance', 'HR'],
    'Gender': ['Male', 'Female', 'Male', 'Female'],
    'Experience': [3, 5, 7, 2],
    'Salary': [6.5, 9.0, 12.0, 5.5]
})

# Step 2: Separate features and target
X = df[['Department', 'Gender', 'Experience']]
y = df['Salary']

# Step 3: Define ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(drop='first'), ['Department', 'Gender']),  # categorical
        ('num', StandardScaler(), ['Experience'])  # numerical
    ]
)

# Step 4: Fit and transform the data
X_transformed = preprocessor.fit_transform(X)

# Step 5: Train a simple Linear Regression model
model = LinearRegression()
model.fit(X_transformed, y)

print("Transformed Feature Shape:", X_transformed.shape)
print("Model Coefficients:", model.coef_)
print("Model Intercept:", model.intercept_)


Transformed Feature Shape: (4, 4)
Model Coefficients: [-0.88904899 -0.84726225 -0.15273775  2.21358667]
Model Intercept: 8.982708933717579
